In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from PIL import Image, ImageEnhance
import os
import shutil
import hashlib
import numpy as np

In [3]:
data_path = "/kaggle/working/"

# if (os.path.exists(data_path + "original_dataset/") == False):
#   os.mkdir(data_path + "original_dataset/")
original_data_path = data_path + "original_dataset/"

if (os.path.exists(data_path + "augmented_dataset/") == False):
  os.mkdir(data_path + "augmented_dataset/")
augmented_data_path = data_path + "augmented_dataset/"

In [4]:
# !unzip -n "/content/drive/MyDrive/Colab Notebooks/MRI augmentation/datasets/Data.zip" -d "/content/drive/MyDrive/Colab Notebooks/MRI augmentation/datasets/original_dataset/"

In [5]:
# original_data_path = original_data_path + "Data/"

In [6]:
shutil.copytree('/kaggle/input/brain-mri-initial/initial', original_data_path)

'/kaggle/working/original_dataset/'

In [7]:
for subdir, dirs, files in os.walk(original_data_path):
    for file in dirs:
        print(file)

no_tumor
glioma_tumor
pituitary_tumor
meningioma_tumor


In [8]:
hashes = set()

for subdir, dirs, files in os.walk(original_data_path):
  for directory in dirs:
    removed = 0
    for filename in os.listdir(original_data_path + directory):
      path = os.path.join(original_data_path, directory, filename)
      digest = hashlib.sha1(open(path,'rb').read()).digest()
      if digest not in hashes:
          hashes.add(digest)
      else:
          os.remove(path)
          removed = removed + 1
    print("Duplicate images removed from " + directory + ": " + str(removed))

Duplicate images removed from no_tumor: 148
Duplicate images removed from glioma_tumor: 24
Duplicate images removed from pituitary_tumor: 55
Duplicate images removed from meningioma_tumor: 10


In [9]:
def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):
  # Convert image to NumPy array
  img_array = np.array(image)

  # Get image shape
  rows, cols = img_array.shape

  # Generate random noise mask
  salt_mask = np.random.rand(rows, cols) < salt_prob
  pepper_mask = np.random.rand(rows, cols) < pepper_prob

  # Add salt noise (white pixels)
  img_array[salt_mask] = 255

  # Add pepper noise (black pixels)
  img_array[pepper_mask] = 0

  # Convert back to PIL Image
  noisy_image = Image.fromarray(img_array)

  return noisy_image

In [10]:
def histogram_equalization(image):
  # Convert image to NumPy array
  img_array = np.asarray(image)

  #flatten image array and calculate histogram via binning
  histogram_array = np.bincount(img_array.flatten(), minlength=256)

  #normalize
  num_pixels = np.sum(histogram_array)
  histogram_array = histogram_array/num_pixels

  #normalized cumulative histogram
  chistogram_array = np.cumsum(histogram_array)

  #Pixel mapping lookup table
  transform_map = np.floor(255 * chistogram_array).astype(np.uint8)

  # flatten image array into 1D list
  img_list = list(img_array.flatten())

  # transform pixel values to equalize
  eq_img_list = [transform_map[p] for p in img_list]

  # reshape and write back into img_array
  eq_img_array = np.reshape(np.asarray(eq_img_list), img_array.shape)

  #convert NumPy array to pillow Image and write to file
  equalized_image = Image.fromarray(eq_img_array, mode='L')

  return equalized_image

In [11]:
baseheight = 224
width = 224

for subdir, dirs, files in os.walk(original_data_path):
  for directory in dirs:
    i = 0
    if (os.path.exists(augmented_data_path + directory + "/") == False):
      os.mkdir(augmented_data_path + directory + "/")
    for item in os.listdir(original_data_path + directory):
      img = Image.open(original_data_path + directory + "/" + item).convert('L') # Converted to grayscale
      save = img.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ").jpg")

      salt_probability = 0.03  # adjust as needed
      pepper_probability = 0.03  # adjust as needed
      noisy_img_SP = add_salt_and_pepper_noise(img, salt_probability, pepper_probability) # Add salt and pepper noise to the image
      noisy_img_SP = noisy_img_SP.resize((width, baseheight), Image.Resampling.LANCZOS)
      save_SP = noisy_img_SP.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_SP.jpg")

      equalized_img_HE = histogram_equalization(img) # Perform histogram equalization
      equalized_img_HE = equalized_img_HE.resize((width, baseheight), Image.Resampling.LANCZOS)
      save_HE = equalized_img_HE.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_HE.jpg")

      rotate_img_RA = img.rotate(10)
      rotate_img_RC = img.rotate(-10)
      rotate_img_RA = rotate_img_RA.resize((width, baseheight), Image.Resampling.LANCZOS)
      rotate_img_RC = rotate_img_RC.resize((width, baseheight), Image.Resampling.LANCZOS)
      save_RA = rotate_img_RA.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_RA.jpg")
      save_RC = rotate_img_RC.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_RC.jpg")

      vertical_img_VF = img.transpose(method=Image.FLIP_TOP_BOTTOM)
      horizontal_img_HF = img.transpose(method=Image.FLIP_LEFT_RIGHT)
      vertical_img_VF = vertical_img_VF.resize((width, baseheight), Image.Resampling.LANCZOS)
      horizontal_img_HF = horizontal_img_HF.resize((width, baseheight), Image.Resampling.LANCZOS)
      save_VF = vertical_img_VF.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_VF.jpg")
      save_HF = horizontal_img_HF.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_HF.jpg")

      enhancer = ImageEnhance.Brightness(img)
      bright_factor = 1.5 #brightens the image
      dark_factor = 0.5 #darkens the image
      bright_img_BR = enhancer.enhance(bright_factor)
      dark_img_DA = enhancer.enhance(dark_factor)
      bright_img_BR = bright_img_BR.resize((width, baseheight), Image.Resampling.LANCZOS)
      dark_img_DA = dark_img_DA.resize((width, baseheight), Image.Resampling.LANCZOS)
      save_BR = bright_img_BR.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_BR.jpg")
      save_DA = dark_img_DA.save(augmented_data_path + directory + "/" + directory + "_aug_(" + str(i + 1) + ")_DA.jpg")

      i = i + 1
    print("Total images in " + directory + ": " + str(i * 9))

Total images in no_tumor: 3168
Total images in glioma_tumor: 8118
Total images in pituitary_tumor: 7614
Total images in meningioma_tumor: 8343
